# NBA "Player-Shot" Stats Collection from NBA API
This notebook contains details of how to pull low-level details of every shot a player has taken throughout an NBA season, which can be retrieved by simply using the free NBA-API provided by Swar's NBA API client package. 

<b>Documentation / Credits:</b> https://github.com/swar/nba_api
</br></br>
<b>Installation Instructions:</b>
pip install nba_api

In [2]:
import numpy as np
import pandas as pd

from nba_api.stats.static import players
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.endpoints import playercareerstats

pd.options.display.max_columns = None

In [3]:
def get_player_shotchartdetail(player_name,season_id):
    # player dictionary
    nba_players = players.get_players()
    player_dict = [player for player in nba_players if player["full_name"] == player_name][0]
#     print(player_dict)
    
    #career df
    career = playercareerstats.PlayerCareerStats(player_id=player_dict['id'])
    career_df = career.get_data_frames()[0]    
#     print(career_df)
    
    # team id during the season
    team_id = career_df[career_df['SEASON_ID'] == season_id]['TEAM_ID']
#     print(team_id)
    
    # shotchartdetail endpoint
    shotchartlist = shotchartdetail.ShotChartDetail(team_id=int(team_id), 
                                                   player_id=int(player_dict['id']), 
                                                   season_type_all_star='Regular Season', 
                                                   season_nullable=season_id,
                                                   context_measure_simple="FGA").get_data_frames()
    return shotchartlist[0], shotchartlist[1]

    

In [4]:
#Input the Player's full name and year of th season of interest as parameters to the get_player_shotchartdetail function.
#The function will output 2 df's, 1 containing shot details of the player, and 1 containing league average details
player_shotchart_df, league_avg = get_player_shotchartdetail("LeBron James","2019-20")

In [5]:
#this DF now contains details of every shot Lebron has taken in the season of 2019-2020.
player_shotchart_df

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0021900002,7,2544,LeBron James,1610612747,Los Angeles Lakers,1,11,47,Made Shot,Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,-9,23,1,1,20191022,LAC,LAL
1,Shot Chart Detail,0021900002,21,2544,LeBron James,1610612747,Los Angeles Lakers,1,10,5,Made Shot,Turnaround Fadeaway shot,2PT Field Goal,Mid-Range,Left Side(L),8-16 ft.,14,-144,41,1,1,20191022,LAC,LAL
2,Shot Chart Detail,0021900002,59,2544,LeBron James,1610612747,Los Angeles Lakers,1,6,58,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Center(C),24+ ft.,25,-66,251,1,0,20191022,LAC,LAL
3,Shot Chart Detail,0021900002,129,2544,LeBron James,1610612747,Los Angeles Lakers,1,3,51,Missed Shot,Pullup Jump shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,10,5,109,1,0,20191022,LAC,LAL
4,Shot Chart Detail,0021900002,187,2544,LeBron James,1610612747,Los Angeles Lakers,2,10,48,Made Shot,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,29,220,190,1,1,20191022,LAC,LAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,Shot Chart Detail,0021901310,224,2544,LeBron James,1610612747,Los Angeles Lakers,2,6,37,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,26,-181,189,1,0,20200813,LAL,SAC
1299,Shot Chart Detail,0021901310,226,2544,LeBron James,1610612747,Los Angeles Lakers,2,6,27,Missed Shot,Alley Oop Layup shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,21,4,1,0,20200813,LAL,SAC
1300,Shot Chart Detail,0021901310,244,2544,LeBron James,1610612747,Los Angeles Lakers,2,5,2,Made Shot,Running Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,11,6,1,1,20200813,LAL,SAC
1301,Shot Chart Detail,0021901310,247,2544,LeBron James,1610612747,Los Angeles Lakers,2,4,24,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Center(C),24+ ft.,27,58,270,1,0,20200813,LAL,SAC


In [8]:
#Checking for null values
player_shotchart_df['LOC_X'].isnull().any()

False

In [9]:
# this DF contains the league average %'s, # of FGA, and # of FGM from every shot zone, which can be used to 
# compare against the player's percentages to determine if he is above/below league average.

league_avg

,GRID_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,FGA,FGM,FG_PCT
0,League Averages,Above the Break 3,Back Court(BC),Back Court Shot,45,6,0.133
1,League Averages,Above the Break 3,Center(C),24+ ft.,14955,5168,0.346
2,League Averages,Above the Break 3,Left Side Center(LC),24+ ft.,20895,7459,0.357
3,League Averages,Above the Break 3,Right Side Center(RC),24+ ft.,19624,6875,0.350
4,League Averages,Backcourt,Back Court(BC),Back Court Shot,405,7,0.017
5,League Averages,In The Paint (Non-RA),Center(C),8-16 ft.,9820,4159,0.424
6,League Averages,In The Paint (Non-RA),Center(C),Less Than 8 ft.,16915,6518,0.385
7,League Averages,In The Paint (Non-RA),Left Side(L),8-16 ft.,1770,714,0.403
8,League Averages,In The Paint (Non-RA),Right Side(R),8-16 ft.,1871,738,0.394
9,League Averages,Left Corner 3,Left Side(L),24+ ft.,8380,3329,0.397


In [10]:
#Export to Excel
pd.DataFrame.to_excel(player_shotchart_df, "leBron_shotcharts_2019_20.xlsx", index=False)
pd.DataFrame.to_excel(league_avg, "league_avg_2019_20.xlsx", index=False)